In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import sys, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dir = 'data'
cls_count = 5

In [ ]:
generator = {
    'train': ImageDataGenerator(rescale=1. / 255, rotation_range=20, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.1, zoom_range=0.2,
                                horizontal_flip=True, preprocessing_function=preprocess_input).flow_from_directory(directory=dir, target_size=(200, 200),subset='training'),
    'valid': ImageDataGenerator(rescale=1 / 255, validation_split=0.2, preprocessing_function=preprocess_input).flow_from_directory(directory=dir, target_size=(200, 200), subset='validation')
}

In [ ]:
IMG_SIZE = (200, 200, 3)

base_model = VGG16(input_shape=IMG_SIZE, weights='imagenet', include_top=False)
base_model.trainable = False

In [ ]:
x = Flatten()(base_model.output)
x = Dense(cls_count, activation='softmax')(x)

model = Model(base_model.input, x)
model.summary()

In [6]:
batch_size = 128
epochs = 10

train_generator = generator['train']
valid_generator = generator['valid']
steps_per_epoch = train_generator.n // batch_size
validation_steps = valid_generator.n // batch_size

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
r = model.fit(train_generator, epochs=10, validation_data=valid_generator, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps)

In [8]:
model.save('data')
loaded_model = tf.keras.models.load_model('data')

In [ ]:
plt.plot(r.history['accuracy'], label='accuracy')
plt.plot(r.history['val_accuracy'], label='val_accuracy')
plt.legend()

In [ ]:
[f for f in os.listdir('data')]

In [126]:
def random_image():
  cls = ['Butterfly', 'Dragonfly', 'Grasshopper', 'Ladybird', 'Mosquito']
  dir_path = dir + '/' + np.random.choice(cls) + '/'
  img_file = [dir_path + fname for fname in os.listdir(dir_path)]
  img_file = np.random.choice(img_file)
  return img_file

In [127]:
def predict():
    img = image.load_img(random_image(), target_size=(200, 200))
    img = image.img_to_array(img, dtype=np.uint8)
    img = np.array(img)/255.0

    plt.axis('off')
    plt.imshow(img.squeeze())
    plt.show()

    p = loaded_model.predict(np.expand_dims(img, axis=0))

    labels = {0: 'Butterfly', 1: 'Dragonfly', 2: 'Grasshopper', 3: 'Ladybird', 4: 'Mosquito'}

    print(np.max(p[0], axis=-1), labels[np.argmax(p[0], axis=-1)])

In [ ]:
predict()